In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import os
from sentence_transformers import SentenceTransformer
from collections import namedtuple
from torch.utils.data import random_split
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from mteb import MTEB

In [ ]:
sts_tasks = [
    "STSBenchmark",   # Standard semantic similarity benchmark
    "SICK-R",         # Semantic relatedness from the SICK dataset
    "STS12",
    "STS13",
    "STS14",
    "STS15",
    "STS16",
]

evaluation = MTEB(tasks=sts_tasks)


def eval(eval_model):
    results = evaluation.run(eval_model, output_folder=None)
    score = [result.scores['test'][0]['main_score']
                     for result in results]
    tasks = [result.task_name for result in results]
    # Print list of tasks as comma-separated values
    print(", ".join(tasks))
    # Print list of scores as comma-separated values    
    print(", ".join([f"{score:.4f}" for score in score]))
    return score

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"

In [ ]:


class LLaMAEmbeddingModel:
    def __init__(self, model, tokenizer, normalize_embeddings=True, mean_pooling=True, layer_idx = -1, ignore_bos_token = False, batch_size=8, W_aug=torch.nn.Identity()):
        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()
        # Remove the second half of decoder layers
        # self.model.model.layers = self.model.model.layers[:len(self.model.model.layers)//2]
        self.model.lm_head = torch.nn.Identity()
        self.normalize_embeddings = normalize_embeddings
        self.mean_pooling = mean_pooling
        self.batch_size = batch_size
        self.layer_idx = layer_idx
        self.ignore_bos_token = ignore_bos_token
        self.W_aug = W_aug.cuda()

    def encode(self, sentences, **kwargs):
        all_embeddings = []
        for i in range(0, len(sentences), self.batch_size):
            batch = sentences[i:i+self.batch_size]
            inputs = self.tokenizer(
                batch, return_tensors='pt', padding=True, truncation=True).to(self.model.device)
            with torch.no_grad():
                output = self.model(**inputs, output_hidden_states=True, return_dict=True)
                outputs = output.hidden_states[self.layer_idx]
                # Mean pooling
                if self.mean_pooling:
                    attention_mask = inputs['attention_mask']
                    embeddings = outputs * attention_mask.unsqueeze(-1)
                    if self.ignore_bos_token:
                        # Ignore the first token (BOS)
                        embeddings = embeddings[:, 1:, :]

                    embeddings = embeddings.mean(1)
                else:
                    # get last token embedding
                    embeddings = outputs[:, -1, :]
                    # Apply ridge regression projection
                    embeddings = self.W_aug(embeddings)

                if self.normalize_embeddings:
                    embeddings = torch.nn.functional.normalize(
                        embeddings, p=2, dim=1)
                all_embeddings.append(embeddings.cpu().float().numpy())
        return np.vstack(all_embeddings)

In [ ]:
model_name = 'meta-llama/Llama-2-7b-hf'
model = AutoModelForCausalLM.from_pretrained(model_name,  device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [ ]:

eval_model = LLaMAEmbeddingModel(
    model, tokenizer, normalize_embeddings=False, mean_pooling=False)
print(eval(eval_model))

In [ ]:
eval_model = LLaMAEmbeddingModel(
    model, tokenizer, normalize_embeddings=False, mean_pooling=True)
print(eval(eval_model))

In [ ]:
n_layers = len(model.model.layers)

eval_model = LLaMAEmbeddingModel(
    model, tokenizer, normalize_embeddings=False, mean_pooling=True, ignore_bos_token=False, layer_idx=19)
print(eval(eval_model))

In [ ]:
eval_model = SentenceTransformer("all-MiniLM-L6-v2")
print(eval(eval_model))

In [ ]:
eval_model = SentenceTransformer("all-mpnet-base-v2")
print(eval(eval_model))

In [ ]:
model = SentenceTransformer("WhereIsAI/UAE-Large-V1")
print(eval(model))